In [ ]:
import numpy as np
from scipy.fft import fft, ifft
from scipy.special import hermite
from scipy.special import factorial
from scipy.linalg import eigh as eig
import matplotlib
from matplotlib import pyplot as plt
import matplotlib.animation as anim
from IPython.display import HTML
import time
import gc
%matplotlib qt
%load_ext autoreload
%autoreload 2

matplotlib.rcParams['animation.embed_limit']=2**128

#Settings to make graph look nice
SMALL_SIZE = 10
MEDIUM_SIZE = 15
BIGGER_SIZE = 20
plt.rc('font', size=BIGGER_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title


def sort_eigen(eigenval,eigenvec):
    #scipy eigh may not return eigenvectors and eigenvals sorted by eigenval
    sorted_index=eigenval.argsort()
    eigenval=eigenval[sorted_index]
    eigenvec=eigenvec[sorted_index]
    
    return eigenval,eigenvec

def time_evol_step(psi_0,UT,X,t,a,b,c):
    UV=split_position(X,t,a,b,c)
    psi_1=UV*psi_0
    phi_1=fft(psi_1)/np.sqrt(N) #to deal with normalisation
    phi_2=UT*phi_1
    psi_2=ifft(phi_2*np.sqrt(N))
    psi_3=UV*psi_2
    
    return psi_3

def time_evol(psi_0,UT,X,a,b,c,steps):
    for i in range(0,steps+1):
        psi_0=time_evol_step(psi_0,UT,X,i*dt,a,b,c)
        
    return psi_0

def HO_state(x,n):
    y=np.exp(-x**2/2)*hermite(n)(x)
    normalisation=(np.sum(np.abs(y)**2))**(1/2)
    
    return y/normalisation

def potential(x):
    return 1/2*x**2

def pert_space(x,a,c):
    return a*np.sin(c*x)

def pert_time(t,b):
    return np.cos(b*t)
    
def pert(x,t,a,b,c):
    return pert_space(x,a,c)*pert_time(t,b)

def split_position(x,t,a,b,c):
    return np.exp(-1j*(potential(X)+pert(X,t,a,b,c))*dt/2)

#UT/split_momentum is a constant, so we don't need to write it as a function

In [ ]:
#Parameters for FFT
a=-20 #left end point
b=20 #Right end point
L=b-a #Width of the trap
N=2**12 #Number of cells
N_DVR=N-1
X=a+L/N*np.arange(0,N) #Dimensonless grid coordinate for FFT
X_DVR=a+L/N*np.arange(1,N)
P=2*np.pi/L*np.concatenate((np.arange(0,N/2,),np.arange(-N/2,0))) #translated momentum coordinate
dt=5000/10**6 #time step, we fix the time step
T=800 #time duraction of evolution
M=int(T//dt)+1 #steps per period, +1 in case
A=0.1 #pertubation amplitude
omega=0.4995 #perturbation frequency
wavelength=1 #perturbation wavelength

split_momentum=np.exp(-1j*P**2/2*dt)

In [ ]:
#finding probability of being in different eigenstate
#generates probabilites for multiple eigenstates at once
#but only plot one due to the difference in magnitude of probability

max_eigenstate=5
psi_0=np.asarray(HO_state(X,0))
psi_all=[]
for i in np.arange(max_eigenstate):
    psi_all.append(HO_state(X,i))
psi_all=np.asarray(psi_all)

fig,ax=plt.subplots(figsize=(10,10),facecolor='white',tight_layout=True)
#ax.set_xlim((0,800))
#ax.set_ylim((0,1.1))
ax.set_xlabel('Time')
ax.set_ylabel('Transition probability')
ax.set_title(u'Ground to 2nd eigenstate transition probability (A={},\u03C9={})'.format(A,omega))

t0=time.time()
prob=[np.abs(np.matmul(psi_all,psi_0))**2]
for i in range(0,M):
    psi_0=time_evol_step(psi_0,split_momentum,X,i*dt,A,omega,wavelength)
    prob.append(np.abs(np.matmul(psi_all,psi_0))**2)
t1=time.time()
print("Took {} time steps".format(i+1))
print("Took {:.2f} to generate probabilities".format(t1-t0))

prob=np.asarray(prob)
plot_time=np.arange(0,M+1)*dt
"""
for i in range(4,max_eigenstate):
    ax.plot(np.arange(0,M+1)*dt,prob[:,i])
"""
ax.plot(np.arange(0,M+1)*dt,prob[:,1])
#ax.plot(np.arange(0,M+1)*dt,prob[:,1]*1000)
#ax.plot(np.arange(0,M+1)*dt,prob[:,2]*1000)
plt.show()

In [ ]:
#used to plot for rest of probabilites generated in earlier cell
fig,ax=plt.subplots(figsize=(10,10),facecolor='white',tight_layout=True)
#ax.set_xlim((0,800))
#ax.set_ylim((0,1.1))
ax.set_xlabel('Time')
ax.set_ylabel('Transition probability')
ax.set_title(u'Ground to 2nd eigenstate transition probability (A={},\u03C9={})'.format(A,omega))
ax.plot(np.arange(0,M+1)*dt,prob[:,4])
#ax.plot(np.arange(0,M+1)*dt,prob[:,1]*1000)
#ax.plot(np.arange(0,M+1)*dt,prob[:,2]*1000)
plt.show()

In [ ]:
#First order perturbation
m=0
n=1
psi_m=HO_state(X,m) #initial state
psi_n=HO_state(X,n) #final state
omega_nm=n-m
space_pert_matrix=np.diag(pert_space(X,A,wavelength))
#alternative, but doesn't scale well beyond first order
#space_inner=(np.sum(psi_n*pert_space(X,A,wavelength)*psi_m))**2 
space_inner=np.abs(np.dot(psi_n,np.dot(space_pert_matrix,psi_m)))**2

time_steps=np.arange(0,M+1)*dt
phase_absorption=(np.exp(1j*(omega_nm-omega)*time_steps)-1)/(omega_nm-omega)
phase_emission=(np.exp(1j*(omega_nm+omega)*time_steps)-1)/(omega_nm+omega)
transition_prob_RWA=1/4*space_inner*np.abs(phase_absorption)**2
transition_prob=1/4*space_inner*np.abs(phase_absorption+phase_emission)**2

fig,ax=plt.subplots(figsize=(10,10),facecolor='white',tight_layout=True)
#ax.set_xlim((0,800))
#ax.set_ylim((0,1.1))
ax.set_xlabel('Time')
ax.set_ylabel('Transition probability')
ax.set_title(u'Ground to 2nd eigenstate transition probability (A={},\u03C9={})'.format(A,omega))
#ax.plot(np.arange(0,M+1)*dt,np.asarray(prob)[A_index,omega_index,:,1],label='exact')
ax.plot(time_steps,prob[:,1],label='exact')
ax.plot(time_steps,transition_prob,label='1st order')
#ax.plot(time_steps,transition_prob_RWA,label='1st order (RWA)')
ax.legend()
plt.show()

In [ ]:
#generalisation to queue for a range of A and omega
range_A=1/(10**np.arange(1,6)) #range of A to test
range_omega=(1-1/(10**(np.arange(5,0,-1)))) #range of omega to test
print(range_A)
print(range_omega)

min_eigenstate=0
max_eigenstate=5
psi_all=[]
for i in np.arange(min_eigenstate,max_eigenstate):
    psi_all.append(HO_state(X,i))
psi_all=np.asarray(psi_all)

psi_0=HO_state_state(X,0)

t0=time.time()
prob=[]
for i in range_A:
    temp1=[]
    for j in range_omega:
        psi_0=np.asarray(HO_state(X,min_eigenstate))
        temp2=[np.abs(np.matmul(psi_all,psi_0))**2]
        print(i,j)
        for k in range(0,M):
            psi_0=time_evol_step(psi_0,split_momentum,X,k*dt,i,j,wavelength)
            temp2.append(np.abs(np.matmul(psi_all,psi_0))**2) 
        temp1.append(temp2)
    prob.append(temp1)

prob=np.asarray(prob) #first 2 indices select A and omega, 3rd index is time step, 4th index is final wavefunction
t1=time.time()
print("Took {:.2f}s to generate probabilities".format(t1-t0))

In [ ]:
A_index=0
omega_index=2
fig,ax=plt.subplots(figsize=(10,10),facecolor='white',tight_layout=True)
ax.set_xlabel('Time')
ax.set_ylabel('Transition probability')
ax.set_title(u'ground to 1st eigenstate transition probability (A={},\u03C9={})'.format(range_A[A_index],range_omega[omega_index]))

plot_time=np.arange(0,M+1)*dt
ax.plot(np.arange(0,M+1)*dt,np.asarray(prob)[A_index,omega_index,:,1])
#ax.plot(np.arange(0,M+1)*dt,prob[:,1]*1000)
#ax.plot(np.arange(0,M+1)*dt,prob[:,2]*1000)
plt.show()

In [ ]:
#First order perturbation
A_index=2
omega_index=3
m=0
n=1
psi_m=HO_state(X,m) #initial state
psi_n=HO_state(X,n) #final state
omega_nm=n-m
space_pert_matrix=np.diag(pert_space(X,range_A[A_index],wavelength))
#alternative, but doesn't scale well beyond first order
#space_inner=(np.sum(psi_n*pert_space(X,A,wavelength)*psi_m))**2 
space_inner=np.abs(np.dot(psi_n,np.dot(space_pert_matrix,psi_m)))**2

time_steps=np.arange(0,M+1)*dt
phase_absorption=(np.exp(1j*(omega_nm-range_omega[omega_index])*time_steps)-1)/(omega_nm-range_omega[omega_index])
phase_emission=(np.exp(1j*(omega_nm+range_omega[omega_index])*time_steps)-1)/(omega_nm+range_omega[omega_index])
transition_prob_RWA=1/4*space_inner*np.abs(phase_absorption)**2
transition_prob=1/4*space_inner*np.abs(phase_absorption+phase_emission)**2

fig,ax=plt.subplots(figsize=(10,10),facecolor='white',tight_layout=True)
#ax.set_xlim((0,800))
#ax.set_ylim((0,1.1))
ax.set_xlabel('Time')
ax.set_ylabel('Transition probability')
ax.set_title(u'Ground to 1st eigenstate transition probability (A={},\u03C9={})'.format(range_A[A_index],range_omega[omega_index]))
ax.plot(np.arange(0,M+1)*dt,np.asarray(prob)[A_index,omega_index,:,1],label='exact')
ax.plot(time_steps,transition_prob,label='1st order')
#ax.plot(time_steps,transition_prob_RWA,label='1st order (RWA)')
ax.legend()
plt.show()

In [ ]:
#First order perturbation
m=0
n=2
psi_m=HO_state(X,m) #initial state
psi_n=HO_state(X,n) #final state
omega_nm=n-m

for A_index in range(len(range_A)):
    for omega_index in range(len(range_omega)):
        space_pert_matrix=np.diag(pert_space(X,range_A[A_index],wavelength))
        #alternative, but doesn't scale well beyond first order
        #space_inner=(np.sum(psi_n*pert_space(X,A,wavelength)*psi_m))**2 
        space_inner=np.abs(np.dot(psi_n,np.dot(space_pert_matrix,psi_m)))**2

        time_steps=np.arange(0,M+1)*dt
        phase_absorption=(np.exp(1j*(omega_nm-range_omega[omega_index])*time_steps)-1)/(omega_nm-range_omega[omega_index])
        phase_emission=(np.exp(1j*(omega_nm+range_omega[omega_index])*time_steps)-1)/(omega_nm+range_omega[omega_index])
        transition_prob_RWA=1/4*space_inner*np.abs(phase_absorption)**2
        transition_prob=1/4*space_inner*np.abs(phase_absorption+phase_emission)**2

        fig,ax=plt.subplots(figsize=(10,10),facecolor='white',tight_layout=True)
        #ax.set_xlim((0,800))
        #ax.set_ylim((0,1.1))
        ax.set_xlabel('Time')
        ax.set_ylabel('Transition probability')
        ax.set_title(u'Ground to 2nd eigenstate transition probability (A={},\u03C9={})'.format(range_A[A_index],range_omega[omega_index]))
        ax.plot(np.arange(0,M+1)*dt,np.asarray(prob)[A_index,omega_index,:,n],label='exact')
        ax.plot(time_steps,transition_prob,label='1st order')
        #ax.plot(time_steps,transition_prob_RWA,label='1st order (RWA)')
        ax.legend()
        plt.show()

In [ ]:
fig,ax=plt.subplots(figsize=(10,10),facecolor='white',tight_layout=True)
#ax.set_xlim((0,800))
#ax.set_ylim((0,1.1))
ax.set_xlabel('Time')
ax.set_ylabel('Transition probability')
ax.set_title(u'Ground to 1st eigenstate transition probability (A={},\u03C9={})'.format(A,omega))
ax.plot(time_steps,prob[:,1]-transition_prob_RWA)
plt.show()

In [ ]:
#Second order perturbation
m=0
n=2
psi_m=HO_state(X,m) #initial state
psi_n=HO_state(X,n) #final state
omega_nm=n-m
space_pert_matrix=np.diag(pert_space(X,A,wavelength))

#phase factors
time_steps=np.arange(0,M+1)*dt
term_a=-(np.exp(1j*(omega_nm+2*omega)*time_steps)-1)/(omega_nm+2*omega)
term_c=-(np.exp(1j*omega_nm*time_steps)-1)/omega_nm
term_e=-(np.exp(1j*omega_nm*time_steps)-1)/(omega_nm)
term_f=(np.exp(1j*(omega_nm+omega)*time_steps)-1)/(omega_nm+omega)
term_g=-(np.exp(1j*(omega_nm-2*omega)*time_steps)-1)/(omega_nm-2*omega)
term_h=(np.exp(1j*(omega_nm-omega)*time_steps)-1)/(omega_nm-omega)

temp_transition_prob_RWA=[]
temp_transition_prob=[]
t0=time.time()
#summing over all states
num_states=150
for i in range(0,num_states):
    psi_intermediate=HO_state(X,i)
    space_inner=np.dot(psi_n,np.dot(space_pert_matrix,psi_intermediate))*np.dot(psi_intermediate,np.dot(space_pert_matrix,psi_m))
    phase_emission=term_a*1/(i-m+omega)
    phase_absorption=term_g*1/(i-m-omega)
    term_b=(np.exp(1j*(n-i+omega)*time_steps)-1)/((i-m+omega)*(n-i+omega))
    term_d=(np.exp(1j*(n-i-omega)*time_steps)-1)/((i-m+omega)*(n-i-omega))
    temp_transition_prob.append(1/4*space_inner*(term_a/(i-m+omega)+term_b+
    term_c/(i-m+omega)+term_d+term_e/(i-m-omega)+term_f/(i-m-omega)+term_g/(i-m-omega)))
    temp_transition_prob_RWA.append(1/4*space_inner*phase_absorption)
print("Generating probabilites took {}s".format(time.time()-t0))


temp_transition_prob=np.transpose(np.asarray(temp_transition_prob))
temp_transition_prob_RWA=np.transpose(np.asarray(temp_transition_prob_RWA))
ones=np.ones(num_states)
transition_prob_RWA=np.abs(np.dot(temp_transition_prob_RWA,ones))**2
transition_prob=np.abs(np.dot(temp_transition_prob,ones))**2

fig,ax=plt.subplots(figsize=(10,10),facecolor='white',tight_layout=True)
#ax.set_xlim((0,800))
#ax.set_ylim((0,1.1))
ax.set_xlabel('Time')
ax.set_ylabel('Transition probability')
ax.set_title(u'Ground to 2nd eigenstate transition probability (A={},\u03C9={})'.format(A,omega))
ax.plot(np.arange(0,M+1)*dt,prob[:,2],label='exact')
ax.plot(time_steps,transition_prob,label='2nd order')
ax.plot(time_steps,transition_prob_RWA,label='2nd order (RWA)')
ax.legend()
plt.show()

In [ ]:
#Second order perturbation
m=0
n=2
psi_m=HO_state(X,m) #initial state
psi_n=HO_state(X,n) #final state
omega_nm=n-m

for A_index in range(len(range_A)):
    for omega_index in range(len(range_omega)):
        
        space_pert_matrix=np.diag(pert_space(X,range_A[A_index],wavelength))
        
        #phase factors
        time_steps=np.arange(0,M+1)*dt
        term_a=-(np.exp(1j*(omega_nm+2*omega)*time_steps)-1)/(omega_nm+2*omega)
        term_c=-(np.exp(1j*omega_nm*time_steps)-1)/omega_nm
        term_e=-(np.exp(1j*omega_nm*time_steps)-1)/(omega_nm)
        term_f=(np.exp(1j*(omega_nm+omega)*time_steps)-1)/(omega_nm+omega)
        term_g=-(np.exp(1j*(omega_nm-2*omega)*time_steps)-1)/(omega_nm-2*omega)
        term_h=(np.exp(1j*(omega_nm-omega)*time_steps)-1)/(omega_nm-omega)
        
        temp_transition_prob_RWA=[]
        temp_transition_prob=[]
        #summing over all states
        num_states=150
        for i in range(0,num_states):
            psi_intermediate=HO_state(X,i)
            space_inner=np.dot(psi_n,np.dot(space_pert_matrix,psi_intermediate))*np.dot(psi_intermediate,np.dot(space_pert_matrix,psi_m))
            phase_emission=term_a*1/(i-m+omega)
            phase_absorption=term_g*1/(i-m-omega)
            term_b=(np.exp(1j*(n-i+omega)*time_steps)-1)/((i-m+omega)*(n-i+omega))
            term_d=(np.exp(1j*(n-i-omega)*time_steps)-1)/((i-m+omega)*(n-i-omega))
            temp_transition_prob.append(1/4*space_inner*(term_a/(i-m+omega)+term_b+
            term_c/(i-m+omega)+term_d+term_e/(i-m-omega)+term_f/(i-m-omega)+term_g/(i-m-omega)))
            temp_transition_prob_RWA.append(1/4*space_inner*phase_absorption)

        temp_transition_prob=np.transpose(np.asarray(temp_transition_prob))
        temp_transition_prob_RWA=np.transpose(np.asarray(temp_transition_prob_RWA))
        ones=np.ones(num_states)
        transition_prob_RWA=np.abs(np.dot(temp_transition_prob_RWA,ones))**2
        transition_prob=np.abs(np.dot(temp_transition_prob,ones))**2

        fig,ax=plt.subplots(figsize=(10,10),facecolor='white',tight_layout=True)
        #ax.set_xlim((0,800))
        #ax.set_ylim((0,1.1))
        ax.set_xlabel('Time')
        ax.set_ylabel('Transition probability')
        ax.set_title(u'Ground to 2nd eigenstate transition probability (A={},\u03C9={})'.format(range_A[A_index],range_omega[omega_index]))
        ax.plot(np.arange(0,M+1)*dt,np.asarray(prob)[A_index,omega_index,:,n],label='exact')
        ax.plot(time_steps,transition_prob,label='2nd order')
        #ax.plot(time_steps,transition_prob_RWA,label='2nd order (RWA)')
        ax.legend()
        plt.show()

In [ ]:
#Second order perturbation
#we do time step by step here to double check that the result agrees with array method
#This is much slower and is not recommended for doing actual computation
m=0
n=1
psi_m=HO_state(X,m) #initial state
psi_n=HO_state(X,n) #final state
omega_nm=n-m
space_pert_matrix=np.diag(pert_space(X,A,wavelength))

#phase factors
time_steps=np.arange(0,M+1)*dt

t0=time.time()
transition_prob=[]
transition_prob_RWA=[]
for t in time_steps:
    term_a=-(np.exp(1j*(omega_nm+2*omega)*t)-1)/(omega_nm+2*omega)
    term_c=-(np.exp(1j*omega_nm*t)-1)/omega_nm
    term_e=-(np.exp(1j*omega_nm*t)-1)/(omega_nm)
    term_f=(np.exp(1j*(omega_nm+omega)*t)-1)/(omega_nm+omega)
    term_g=-(np.exp(1j*(omega_nm-2*omega)*t)-1)/(omega_nm-2*omega)
    term_h=(np.exp(1j*(omega_nm-omega)*t)-1)/(omega_nm-omega)

    temp_transition_prob_RWA=[]
    temp_transition_prob=[]

    #summing over all states
    num_states=20
    for i in range(0,num_states):
        psi_intermediate=HO_state(X,i)
        #mat=np.matmul(space_pert_matrix,np.matmul(np.diag(psi_intermediate**2),space_pert_matrix))
        #space_inner=np.abs(np.dot(psi_n,np.dot(mat,psi_m)))**2
        space_inner=np.dot(psi_n,np.dot(space_pert_matrix,psi_intermediate))*np.dot(psi_intermediate,np.dot(space_pert_matrix,psi_m))
        #space_inner=1
        phase_emission=term_a*1/(i-m+omega)
        phase_absorption=term_g*1/(i-m-omega)
        term_b=(np.exp(1j*(n-i+omega)*t)-1)/((i-m+omega)*(n-i+omega))
        term_d=(np.exp(1j*(n-i-omega)*t)-1)/((i-m+omega)*(n-i-omega))
        temp_transition_prob.append(1/4*space_inner*(term_a/(i-m+omega)+term_b+
        term_c/(i-m+omega)+term_d+term_e/(i-m-omega)+term_f/(i-m-omega)+term_g/(i-m-omega)))
        temp_transition_prob_RWA.append(1/4*space_inner*phase_absorption)
    transition_prob.append(np.abs(np.sum(temp_transition_prob))**2)
    transition_prob_RWA.append(np.abs(np.sum(temp_transition_prob_RWA))**2)
            
print("Generating probabilites took {}s".format(time.time()-t0))


fig,ax=plt.subplots(figsize=(10,10),facecolor='white',tight_layout=True)
#ax.set_xlim((0,800))
#ax.set_ylim((0,1.1))
ax.set_xlabel('Time')
ax.set_ylabel('Transition probability')
ax.set_title(u'Ground to 1st eigenstate transition probability (A={},\u03C9={})'.format(A,omega))
#ax.plot(np.arange(0,M+1)*dt,100*prob[:,1],label='exact')
ax.plot(time_steps,transition_prob,label='2nd order')
ax.plot(time_steps,transition_prob_RWA,label='2nd order (RWA)')
ax.legend()
plt.show()

In [ ]:
fig,ax=plt.subplots(figsize=(10,10),facecolor='white',tight_layout=True)
#ax.set_xlim((0,800))
#ax.set_ylim((0,1.1))
ax.set_xlabel('Time')
ax.set_ylabel('Transition probability')
ax.set_title(u'Ground to 1st eigenstate transition probability (A={},\u03C9={})'.format(A,omega))
#ax.plot(np.arange(0,M+1)*dt,100*prob[:,1],label='exact')
ax.plot(time_steps,transition_prob,label='2nd order')
#ax.plot(time_steps,transition_prob_RWA,label='2nd order (RWA)')
ax.legend()
plt.show()

In [ ]:
#Parameters for testing adiabaticity
a=-10 #left end point
b=10 #Right end point
L=b-a #Width of the trap
N=2**10 #Number of cells
N_DVR=N-1
X=a+L/N*np.arange(0,N) #Dimensonless grid coordinate for FFT
X_DVR=a+L/N*np.arange(1,N)
P=2*np.pi/L*np.concatenate((np.arange(0,N/2,),np.arange(-N/2,0))) #translated momentum coordinate
A=0.1 #pertubation amplitude
omega=0.01 #perturbation frequency, period is 2pi/omega
wavelength=1 #perturbation wavelength
dt=np.pi/100 #time step, we fix the time step
T=10*2*np.pi #time duraction of evolution
M=int(T//dt)+1 #steps per period, +1 in case

split_momentum=np.exp(-1j*P**2/2*dt)

In [ ]:
#generating K matrix takes a good amount of time, we will generate it seperately to prevent having to generate it everytime we run the code
#Only needs to be generated again if we change our a,b and N conditions, but unlikely that we will need to
t0=time.time()
#kinetic matrix
K=np.zeros((N_DVR,N_DVR))
c=1/2*(np.pi/L)**2*2/(N_DVR+1)
n=np.arange(1,N) #summation index in DVR T matrix element
for i in range(N_DVR):
    for j in range(N_DVR):
        K[i,j]=c*np.sum(n**2*np.sin(n*np.pi*(i+1)/N)*np.sin(n*np.pi*(j+1)/N))
        K[j,i]=K[i,j]
t1=time.time()
print("It took {}s to generate kinetic matrix".format(t1-t0))

In [ ]:
#testing the limits of adiabaticity
V=np.diag(potential(X_DVR)+pert(X_DVR,0,A,omega,wavelength))
E,vec=eig(K+V)
E,vec=sort_eigen(E,vec)
exact_vec=[vec]
exact_E_diff=[E[1]-E[0]]
#we need to add the 0 here to make it match FFT definition
#we also start from the eigenstate of the exact hamiltonian instead of the HO
psi_0=np.insert(vec[:,0],0,0) 

fig,ax=plt.subplots(figsize=(10,10),facecolor='white',tight_layout=True)
#ax.set_xlim((0,800))
#ax.set_ylim((0,1.1))
ax.set_xlabel('Time')
ax.set_ylabel('Probability')
ax.set_title(u'Probability of being in ground state (A={},\u03C9={})'.format(A,omega))

period=2*np.pi/omega
last_period_point=period/dt
period_points=np.arange(period/dt)
print(last_period_point)
#we during the period points, we calculate and store the eigenvectors instead of calculate for all time
#this should speed up our calculations

prob=[1]
E_diff=[E[1]-E[0]]

t0=time.time()
for i in range(0,M):
    if i<=last_period_point:
        V=np.diag(potential(X_DVR)+pert(X_DVR,i*dt,A,omega,wavelength))
        E,vec=eig(K+V)
        E,vec=sort_eigen(E,vec)
        exact_vec.append(vec)
        exact_E_diff.append(E[1]-E[0])
        
    if i%100==0 and i<=last_period_point:
        print(i)
        
    psi_0=time_evol_step(psi_0,split_momentum,X,i*dt,A,omega,wavelength)
    
    E_diff.append(exact_E_diff[int(i%(last_period_point+1))])
    prob.append(np.abs(np.vdot(exact_vec[int(i%(last_period_point+1))][:,0],np.delete(psi_0,1)))**2)
t1=time.time()
print("Took {} time steps".format(i+1))
print("Took {:.2f} to generate probabilities".format(t1-t0))

prob=np.asarray(prob)
plot_time=np.arange(0,M+1)*dt
"""
for i in range(4,max_eigenstate):
    ax.plot(np.arange(0,M+1)*dt,prob[:,i])
"""
ax.plot(plot_time,prob)
plt.show()

In [ ]:
#testing the limits of adiabaticity, generalised for multiple omega and A
omega_range=list(np.arange(1,4))+list(0.1*np.arange(1,10))
A_range=[0.1]
print(A_range)
print(omega_range)
prob=[]
E_diff=[]
t0=time.time()
for l in A_range:
    temp_temp_prob=[]
    temp_temp_E_diff=[]
    for k in omega_range:
        print(l,k)
        V=np.diag(potential(X_DVR)+pert(X_DVR,0,l,k,wavelength))
        E,vec=eig(K+V)
        E,vec=sort_eigen(E,vec)
        exact_vec=[vec]
        exact_E_diff=[E[1]-E[0]]
        #we need to add the 0 here to make it match FFT definition
        #we also start from the eigenstate of the exact hamiltonian instead of the HO
        psi_0=np.insert(vec[:,0],0,0)

        period=2*np.pi/k
        last_period_point=period/dt
        print(last_period_point)
        period_points=np.arange(last_period_point)
        #we during the period points, we calculate and store the eigenvectors instead of calculate for all time
        #this should speed up our calculations

        temp_prob=[1]
        temp_E_diff=[E[1]-E[0]]

        for i in range(0,M):
            #if our total time period is more than 1 oscillation, we can drop
            if i<=last_period_point:
                V=np.diag(potential(X_DVR)+pert(X_DVR,i*dt,l,k,wavelength))
                E,vec=eig(K+V)
                E,vec=sort_eigen(E,vec)
                exact_vec.append(vec)
                exact_E_diff.append(E[1]-E[0])

            if i%100==0 and i<=last_period_point:
                print(i)

            psi_0=time_evol_step(psi_0,split_momentum,X,i*dt,l,k,wavelength)
            temp_E_diff.append(exact_E_diff[int(i%(last_period_point+1))])
            temp_prob.append(np.abs(np.vdot(exact_vec[int(i%(last_period_point+1))][:,0],np.delete(psi_0,1)))**2)

        # unsurpringly, storing eigenstates uses up a lot of memory, we clear the eigenstates to prevent our computer from crashing
        del vec
        gc.collect()
        temp_temp_prob.append(temp_prob)
        temp_temp_E_diff.append(temp_E_diff)
    E_diff.append(temp_temp_E_diff)
    prob.append(temp_temp_prob)
    
t1=time.time()
print("Took {:.2f} to generate probabilities".format(t1-t0))

In [ ]:
#plotting energy difference
A_index=2
fig,ax=plt.subplots(figsize=(10,10),facecolor='white',tight_layout=True)
#ax.set_xlim((0,800))
#ax.set_ylim((0,1.1))
ax.set_xlabel('Time')
ax.set_ylabel('Energy difference')
ax.set_title(u'Energy difference for different perturbation(A={})'.format(A_range[A_index]))

plot_time=np.arange(0,M+1)*dt
for j in range(len(omega_range)):
    ax.plot(plot_time,E_diff[A_index][j],label=u'\u03C9={}'.format(omega_range[j]))
ax.legend()
plt.show()

In [ ]:
#plotting transition probabilities
A_index=0
fig,ax=plt.subplots(figsize=(10,10),facecolor='white',tight_layout=True)
#ax.set_xlim((0,800))
#ax.set_ylim((0,1.1))
ax.set_xlabel('Time')
ax.set_ylabel('Probability')
ax.set_title(u'Probability of being in ground state (A={})'.format(A_range[A_index]))

plot_time=np.arange(0,M+1)*dt
for i in range(3,12):
    if i==2:
        continue
    ax.plot(plot_time,prob[A_index][i],label=u'\u03C9={}'.format(omega_range[i]))
ax.legend()
plt.show()

In [ ]:
#Animation to show how the pertubation changes with time
fig,ax=plt.subplots(figsize=(10,10),facecolor='white')
ax.set_xlim((X[0]-5,X[-1]+5))
ax.set_ylim((-1.2,1.2))
time_text = ax.text(0.05, 0.9, '', transform=ax.transAxes)
#initial=ax.plot(X,np.abs(psi)**2,c='r')
line, = ax.plot([], [], lw=2)
#ax.plot(X,potential(X),c='k')

# animation function. This is called sequentially
def animate(i):
    #y=time_evol(psi_initial,UV,UT,i+1)
    #line.set_data(X, np.abs(y)**2)
    
    line.set_data(X,pert(X,i*dt,A,omega,wavelength))
    time_template = 'time = %.2fT'
    time_text.set_text(time_template % (i/M))
    return (line,)

ani=anim.FuncAnimation(fig,animate,frames=K*M+1,interval=50, blit=True)
HTML(ani.to_jshtml())
HTML(ani.to_html5_video())

In [ ]:
#Warning: Animations take a long time to generate
#This code generates the wavefunction for all time first and then generates the animation
t0=time.time()
psi_0=HO_state(X,0)
fig,ax=plt.subplots(figsize=(10,10),facecolor='white')
ax.set_xlim((X[0]-5,X[-1]+5))
ax.set_ylim((0,0.2))
ax.set_xlabel('x')
ax.set_ylabel('Probability density')
ax.set_title('Probability density over time')
time_text = ax.text(0.05, 0.9, '', transform=ax.transAxes)
initial=ax.plot(X,np.abs(psi_0)**2,c='r')
line, = ax.plot([], [], lw=2)
#ax.plot(X,potential(X),c='k')

#generate wavefunctions for all time first
psi=[psi_0]
for i in range(1,K*M+1):
    psi.append(time_evol_step(psi[-1],split_momentum,X,i*dt,A,omega,wavelength))
print(time.time()-t0)
    
print("generated wavefunctions")
# animation function. This is called sequentially
def animate(i):
    line.set_data(X, np.abs(psi[i])**2)
    time_template = 'time = %.2fT'
    time_text.set_text(time_template % (i/M))
    return (line,)


ani=anim.FuncAnimation(fig,animate,frames=K*M+1,interval=50, blit=True)
#HTML(ani.to_html5_video())
print(time.time()-t0)
HTML(ani.to_jshtml())

In [ ]:
#Warning: Animations take a long time to generate
#This code generates the wavefunction and animation at the same time, but due to the way the animation is implemented in Python,
#i.e. using functions, we need to generate the wavefunction for the animation frame from the starting wavefunction
#This is much slower compared to earlier approach where you generate the wavefunction first with maybe the trade off being the amount
#of RAM to store the info first
psi_0=HO_state(X,0)
t0=time.time()
fig,ax=plt.subplots(figsize=(10,10),facecolor='white')
ax.set_xlim((X[0]-5,X[-1]+5))
ax.set_ylim((0,0.2))
ax.set_xlabel('x')
ax.set_ylabel('Probability density')
ax.set_title('Probability density over time')
time_text = ax.text(0.05, 0.9, '', transform=ax.transAxes)
initial=ax.plot(X,np.abs(psi_0)**2,c='r')
line, = ax.plot([], [], lw=2)
#ax.plot(X,potential(X),c='k')

# animation function. This is called sequentially
def animate(i):
    y=time_evol(psi_0,split_momentum,X,A,omega,wavelength,i)
    line.set_data(X, np.abs(y)**2)
    time_template = 'time = %.2fT'
    time_text.set_text(time_template % (i/M))
    return (line,)

ani=anim.FuncAnimation(fig,animate,frames=K*M+1,interval=50, blit=True)
#HTML(ani.to_html5_video())
HTML(ani.to_jshtml())
print(time.time()-t0)

In [ ]:
#Warning: Animations take a long time to generate
#Animation showing the probability of being in different eigenstate as a bar chart
t0=time.time()
max_eigenstate=50
psi_0=np.asarray(HO_state(X,0))
psi_all=[]
for i in range(max_eigenstate):
    psi_all.append(HO_state(X,i))
psi_all=np.asarray(psi_all)

fig,ax=plt.subplots(figsize=(10,10),facecolor='white')
ax.set_xlim((-0.5,max_eigenstate))
ax.set_ylim((0,1.1))
ax.set_xlabel('Nth eigenstate')
ax.set_ylabel('Probability of being in the nth eigenstate')
ax.set_title('Probability of being in the nth eigenstate over time')
time_text = ax.text(0.025, 0.95, '', transform=ax.transAxes)

prob=np.abs(np.matmul(psi_all,psi_0))**2
initial=ax.bar(np.arange(0,max_eigenstate),prob,color='r')
bar=ax.bar(np.arange(0,max_eigenstate),prob)

# animation function. This is called sequentially
def animate(i):
    y=time_evol(psi_0,split_momentum,X,A,omega,wavelength,i+1)
    prob=np.abs(np.matmul(psi_all,y))**2
    for j in range(len(prob)):
        bar[j].set_height(prob[j])
    time_template = 'time = %.2fT'
    time_text.set_text(time_template % (i/M))
    return bar

ani=anim.FuncAnimation(fig,animate,frames=K*M,interval=50, blit=True)
HTML(ani.to_jshtml())
print("Animation took {}s to generate".format(time.time()-t0))